In [6]:
import torch
import torch.nn as nn
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=50, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        predictions = self.linear(lstm_out[:, -1])
        return predictions

# Fetch and preprocess data
def fetch_and_preprocess_data(ticker):
    # Fetch historical data for the past 3 months
    data = yf.download(ticker, period="3mo", interval="1d")
    closing_prices = data['Close'].values
    
    # Reshape and scale data
    closing_prices = closing_prices.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(closing_prices)
    
    # Prepare training data
    X_train, y_train = [], []
    for i in range(60, len(scaled_data)):
        X_train.append(scaled_data[i-60:i, 0])
        y_train.append(scaled_data[i, 0])
    
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = torch.from_numpy(X_train).float().view(-1, 60, 1)
    y_train = torch.from_numpy(y_train).float().view(-1, 1)
    
    return X_train, y_train, scaler, closing_prices

# Train the model
def train_model(model, X_train, y_train, epochs=1, lr=0.001):
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for i in range(epochs):
        for seq, labels in zip(X_train, y_train):
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                                 torch.zeros(1, 1, model.hidden_layer_size))

            y_pred = model(seq.unsqueeze(0))
            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()

    return model

# Forecast future prices
def forecast_future_prices(model, scaler, closing_prices):
    model.eval()
    
    # Get the last 60 days of data for forecasting
    last_60_days = closing_prices[-60:].reshape(-1, 1)
    last_60_days_scaled = scaler.transform(last_60_days)
    last_60_days_scaled = torch.from_numpy(last_60_days_scaled).float().view(-1, 60, 1)
    
    # Predict the next 10 days
    forecasted_prices = []
    for _ in range(10):
        with torch.no_grad():
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                                 torch.zeros(1, 1, model.hidden_layer_size))
            predicted_price = model(last_60_days_scaled)
            forecasted_prices.append(predicted_price.item())
            last_60_days_scaled = torch.cat((last_60_days_scaled[:, 1:], predicted_price.view(-1, 1, 1)), dim=1)
    
    forecasted_prices = scaler.inverse_transform(np.array(forecasted_prices).reshape(-1, 1)).flatten()
    
    return forecasted_prices

# Main function to get stock data and forecast
def get_stock_data_and_forecast(ticker):
    X_train, y_train, scaler, closing_prices = fetch_and_preprocess_data(ticker)
    model = LSTMModel()
    trained_model = train_model(model, X_train, y_train, epochs=1)
    forecasted_prices = forecast_future_prices(trained_model, scaler, closing_prices)
    
    return closing_prices.flatten(), forecasted_prices

# Example usage
ticker = 'META'
past_prices, future_forecast = get_stock_data_and_forecast(ticker)
print("Past 3 months' stock prices:", past_prices)
print("Forecasted stock prices for the next 10 days:", future_forecast)


[*********************100%***********************]  1 of 1 completed

Past 3 months' stock prices: [474.35998535 467.04998779 466.82998657 477.48999023 476.98999023
 495.05999756 493.76000977 492.95999146 502.6000061  507.47000122
 508.83999634 504.1000061  504.16000366 506.63000488 499.48999023
 501.70001221 494.77999878 498.91000366 510.6000061  513.11999512
 519.55999756 504.22000122 504.67999268 509.5        509.95999146
 539.90997314 529.32000732 530.         534.69000244 512.70001221
 498.86999512 496.16000366 489.79000854 461.98999023 475.8500061
 476.79000854 487.3999939  488.69000244 461.26998901 453.41000366
 465.70001221 465.70999146 463.19000244 474.82998657 497.73999023
 488.14001465 475.73001099 494.08999634 488.92001343 509.63000488
 517.77001953 515.95001221 528.53997803 526.76000977 537.33001709
 527.41998291 529.2800293  526.72998047 535.15997314 531.92999268
 528.         521.11999512 519.09997559 516.7800293  524.65997314]
Forecasted stock prices for the next 10 days: [449.16970171 448.88896872 448.58521219 448.31400454 448.11914108
 


/Users/vanshikwaghela/miniconda3/envs/env1/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [1]:
import torch
import torch.nn as nn
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, num_layers=2, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True, dropout=0.2)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(self.num_layers, 1, self.hidden_layer_size),
                            torch.zeros(self.num_layers, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        predictions = self.linear(lstm_out[:, -1])
        return predictions

# Fetch and preprocess data
def fetch_and_preprocess_data(ticker):
    # Fetch historical data for the past 3 months
    data = yf.download(ticker, period="6mo", interval="1d")
    closing_prices = data['Close'].values
    
    # Reshape and scale data
    closing_prices = closing_prices.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(closing_prices)
    
    # Prepare training data
    X, y = [], []
    for i in range(60, len(scaled_data)):
        X.append(scaled_data[i-60:i, 0])
        y.append(scaled_data[i, 0])
    
    X, y = np.array(X), np.array(y)
    X = torch.from_numpy(X).float().view(-1, 60, 1)
    y = torch.from_numpy(y).float().view(-1, 1)
    
    return X, y, scaler, closing_prices

# Train the model with early stopping
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, lr=0.001):
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    best_val_loss = float("inf")
    patience, patience_counter = 5, 0  # Early stopping patience

    for epoch in range(epochs):
        model.train()
        for seq, labels in zip(X_train, y_train):
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(model.num_layers, 1, model.hidden_layer_size),
                                 torch.zeros(model.num_layers, 1, model.hidden_layer_size))

            y_pred = model(seq.unsqueeze(0))
            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()

        # Validate
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for seq, labels in zip(X_val, y_val):
                model.hidden_cell = (torch.zeros(model.num_layers, 1, model.hidden_layer_size),
                                     torch.zeros(model.num_layers, 1, model.hidden_layer_size))
                y_pred = model(seq.unsqueeze(0))
                val_loss += loss_function(y_pred, labels).item()
        val_loss /= len(X_val)

        print(f'Epoch {epoch+1}/{epochs}, Loss: {single_loss.item():.6f}, Validation Loss: {val_loss:.6f}')
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print("Early stopping due to no improvement in validation loss.")
            break

    return model

# Forecast future prices
def forecast_future_prices(model, scaler, closing_prices):
    model.eval()
    
    # Get the last 60 days of data for forecasting
    last_60_days = closing_prices[-60:].reshape(-1, 1)
    last_60_days_scaled = scaler.transform(last_60_days)
    last_60_days_scaled = torch.from_numpy(last_60_days_scaled).float().view(-1, 60, 1)
    
    # Predict the next 10 days
    forecasted_prices = []
    for _ in range(10):
        with torch.no_grad():
            model.hidden_cell = (torch.zeros(model.num_layers, 1, model.hidden_layer_size),
                                 torch.zeros(model.num_layers, 1, model.hidden_layer_size))
            predicted_price = model(last_60_days_scaled)
            forecasted_prices.append(predicted_price.item())
            last_60_days_scaled = torch.cat((last_60_days_scaled[:, 1:], predicted_price.view(-1, 1, 1)), dim=1)
    
    forecasted_prices = scaler.inverse_transform(np.array(forecasted_prices).reshape(-1, 1)).flatten()
    
    return forecasted_prices

# Main function to get stock data and forecast
def get_stock_data_and_forecast(ticker):
    X, y, scaler, closing_prices = fetch_and_preprocess_data(ticker)
    
    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LSTMModel()
    trained_model = train_model(model, X_train, y_train, X_val, y_val, epochs=50)
    forecasted_prices = forecast_future_prices(trained_model, scaler, closing_prices)
    
    return closing_prices.flatten(), forecasted_prices

# Example usage
ticker = 'AAPL'
past_prices, future_forecast = get_stock_data_and_forecast(ticker)
print("Past 3 months' stock prices:", past_prices)
print("Forecasted stock prices for the next 10 days:", future_forecast)


[*********************100%***********************]  1 of 1 completed
/Users/vanshikwaghela/miniconda3/envs/env1/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50, Loss: 0.000016, Validation Loss: 0.010854
Epoch 2/50, Loss: 0.002022, Validation Loss: 0.009042
Epoch 3/50, Loss: 0.002986, Validation Loss: 0.009006
Epoch 4/50, Loss: 0.004605, Validation Loss: 0.009000
Epoch 5/50, Loss: 0.000872, Validation Loss: 0.008963
Epoch 6/50, Loss: 0.001384, Validation Loss: 0.008915
Epoch 7/50, Loss: 0.001909, Validation Loss: 0.008861
Epoch 8/50, Loss: 0.001271, Validation Loss: 0.008821
Epoch 9/50, Loss: 0.003561, Validation Loss: 0.008796
Epoch 10/50, Loss: 0.002857, Validation Loss: 0.008666
Epoch 11/50, Loss: 0.000571, Validation Loss: 0.008592
Epoch 12/50, Loss: 0.000552, Validation Loss: 0.008534
Epoch 13/50, Loss: 0.003955, Validation Loss: 0.008966
Epoch 14/50, Loss: 0.003173, Validation Loss: 0.008410
Epoch 15/50, Loss: 0.002597, Validation Loss: 0.008239
Epoch 16/50, Loss: 0.002263, Validation Loss: 0.008107
Epoch 17/50, Loss: 0.002351, Validation Loss: 0.008068
Epoch 18/50, Loss: 0.000418, Validation Loss: 0.007944
Epoch 19/50, Loss: 

In [4]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

# get historical market data
hist = msft.history(period="1mo")

# show meta information about the history (requires history() to be called first)
msft.history_metadata

# show actions (dividends, splits, capital gains)
msft.actions
msft.dividends
msft.splits
msft.capital_gains  # only for mutual funds & etfs

# show share count
msft.get_shares_full(start="2022-01-01", end=None)

# show financials:
msft.calendar
msft.sec_filings
# - income statement
msft.income_stmt
msft.quarterly_income_stmt
# - balance sheet
msft.balance_sheet
msft.quarterly_balance_sheet
# - cash flow statement
msft.cashflow
msft.quarterly_cashflow
# see `Ticker.get_income_stmt()` for more options

# show holders
msft.major_holders
msft.institutional_holders
msft.mutualfund_holders
msft.insider_transactions
msft.insider_purchases
msft.insider_roster_holders

msft.sustainability

# show recommendations
msft.recommendations
msft.recommendations_summary
msft.upgrades_downgrades

# show analysts data
msft.analyst_price_targets
msft.earnings_estimate
msft.revenue_estimate
msft.earnings_history
msft.eps_trend
msft.eps_revisions
msft.growth_estimates

# Show future and historic earnings dates, returns at most next 4 quarters and last 8 quarters by default.
# Note: If more are needed use msft.get_earnings_dates(limit=XX) with increased limit argument.
msft.earnings_dates

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
msft.isin

# show options expirations
msft.options

# show news
msft.news

# get option chain for specific expiration
opt = msft.option_chain('2024-10-25')

,numberOfAnalysts,avg,low,high,yearAgoEps,growth
0q,30,3.10,2.96,3.17,2.73,0.136
+1q,30,3.23,3.07,3.37,2.93,0.102
0y,42,13.07,12.74,13.64,11.80,0.108
+1y,41,15.21,14.23,16.26,13.07,0.164
